# Machine Learning Assignment 2

## Headers

In [1]:
import re

headers = []

with open("datadescription.txt", "r", encoding="utf-8") as file:
    col_num = 1
    for line in file:
        if f"{col_num} - " in line:
            match = re.search("[0-9] - (.*?)\w+", line)
            headers.append(match.group(0)[4:])
            col_num = col_num + 1
        # END IF
    # END FOR
# END WITH

cat_headers = ["job", "marital", "education", "default", "housing", "loan", "contact", "day", "month", "poutcome", "output"]

cont_headers = ["age", "balance", "duration", "campaign", "pdays", "previous"]
print(headers)

['id', 'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'output']


## Creating DataFrame

In [2]:
import pandas as pd

df = pd.read_csv("trainingset.txt", names=headers, na_values="unknown")
df

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,output
0,TR1,44,JobCat9,single,secondary,no,29,yes,no,NaN,5,may,0,1,-1,0,NaN,TypeA
1,TR2,31,JobCat4,married,secondary,no,2,yes,yes,NaN,5,may,0,1,-1,0,NaN,TypeA
2,TR3,42,JobCat4,divorced,tertiary,yes,2,yes,no,NaN,5,may,0,1,-1,0,NaN,TypeA
3,TR4,58,JobCat2,married,primary,no,121,yes,no,NaN,5,may,0,1,-1,0,NaN,TypeA
4,TR5,43,JobCat9,single,secondary,no,593,yes,no,NaN,5,may,0,1,-1,0,NaN,TypeA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24313,TR24314,36,JobCat9,married,secondary,no,557,yes,no,cellular,16,nov,0,4,-1,0,NaN,TypeB
24314,TR24315,53,JobCat6,married,tertiary,no,583,no,no,cellular,17,nov,0,1,184,4,success,TypeB
24315,TR24316,23,JobCat9,single,secondary,no,505,no,yes,cellular,17,nov,0,2,-1,0,NaN,TypeB
24316,TR24317,51,JobCat9,married,tertiary,no,825,no,no,cellular,17,nov,0,3,-1,0,NaN,TypeB


## Functions

In [3]:
import numpy as np
import math

def getPercentage(amount, base, decimal=0):
  return round((amount / base) * 100, decimal)
# END getPercentage

def create_cat_quality_report(df, cat_features):
  features = []
  counts = []
  missing_percentages = []
  cardinalities = []
  modes = []
  mode_freqs = []
  mode_percentages = []
  second_modes = []
  second_mode_freqs = []
  second_mode_percentages = []

  for feature in cat_features:
    curr_feature = df[feature]
    features.append(feature)

    curr_value_counts = curr_feature.value_counts(dropna=False)

    # curr_full_count includes NaN values
    curr_full_count = curr_value_counts.sum()
    counts.append(curr_full_count)

    missing = curr_full_count - curr_feature.count()
    missing_percentages.append(getPercentage(missing, curr_full_count, 1))

    cardinalities.append(len(curr_value_counts.index))

    modes.append(curr_value_counts.index[0])
    mode_freqs.append(curr_value_counts.iloc[0])
    mode_percentages.append(getPercentage(curr_value_counts.iloc[0], curr_full_count, 1))

    second_modes.append(curr_value_counts.index[1])
    second_mode_freqs.append(curr_value_counts.iloc[1])
    second_mode_percentages.append(getPercentage(curr_value_counts.iloc[1], curr_full_count, 1))
  # END FOR

  return {
    "Feature": pd.Series(features),
    "Count": pd.Series(counts),
    "% Miss.": pd.Series(missing_percentages),
    "Cardinality": pd.Series(cardinalities),
    "Mode": pd.Series(modes),
    "Mode Freq.": pd.Series(mode_freqs),
    "Mode %": pd.Series(mode_percentages),
    "2nd Mode": pd.Series(second_modes),
    "2nd Mode Freq.": pd.Series(second_mode_freqs),
    "2nd Mode %": pd.Series(second_mode_percentages)
  }
# END create_cat_quality_report

def create_cont_quality_report(df, cont_features):
  counts = []
  missing_percentages = []
  cardinalities = []
  mins = []
  first_quarts = []
  means = []
  medians = []
  third_quarts = []
  maxes = []
  standard_deviations = []
  
  for feature in cont_features:
    curr_feature_series = cont_df[feature]

    curr_value_counts = curr_feature_series.value_counts(dropna=False)

    # curr_full_count includes NaN values
    curr_full_count = curr_value_counts.sum()
    counts.append(curr_full_count)

    missing = curr_full_count - curr_feature_series.count()
    missing_percentages.append(getPercentage(missing, curr_full_count, 1))

    cardinalities.append(len(curr_value_counts.index))

    mins.append(curr_feature_series.min())

    quantiles = curr_feature_series.quantile([0.25, 0.75])

    first_quarts.append(quantiles[0.25])

    means.append(curr_feature_series.mean()) 

    medians.append(curr_feature_series.median())

    third_quarts.append(quantiles[0.75])

    maxes.append(curr_feature_series.max())

    standard_deviations.append(curr_feature_series.std())
  # END FOR

  return {
    "Feature": pd.Series(cont_features),
    "Count": pd.Series(counts),
    "% Miss.": pd.Series(missing_percentages),
    "Card.": pd.Series(cardinalities),
    "Min": pd.Series(mins),
    "1st Qrt.": pd.Series(first_quarts),
    "Mean": pd.Series(means),
    "Median": pd.Series(medians),
    "3rd Qrt.": pd.Series(third_quarts),
    "Max": pd.Series(maxes),
    "Std. Dev": pd.Series(standard_deviations),
  }
# END create_cont_quality_report()

def getEqualFreq(x, nbin):
    nlen = len(x)
    return np.interp(np.linspace(0, nlen, nbin + 1),
                     np.arange(nlen),
                     np.sort(x))
# END getEqualFreq

def create_pdf_parameters():
    params = {}
    for index, feature in enumerate(cont_quality_report["Feature"]):
        if feature in cont_dropped_headers:
            continue
        # END IF
        
        params[feature] = {"mean": cont_quality_report["Mean"][index], "dev": cont_quality_report["Std. Dev"][index]}
    # END FOR
    
    return params
# END create_pdf_parameters()

def calc_exp_probability(x, mean, feature):
    m = 1 / mean
    if (-1 * mean * x) > 1000:
        print(f"feautre: {feature}, mean: {mean}, x: {x}")
    return m * np.exp(-1 * mean * x)
# END calc_exp_probability()
    
def calc_clamps(feature):
    quantiles = feature.quantile([0.25, 0.5, 0.75])
    lower = quantiles[0.25] - (1.5 * quantiles[0.5])
    upper = quantiles[0.75] + (1.5 * quantiles[0.5])
    return lower, upper
# END calc_clamps

## Categorical Quality Report

### Create categorical quality report

In [4]:
cat_df = df[cat_headers].copy()
cat_df[:5]


cat_quality_report = create_cat_quality_report(df, cat_headers)
pd.DataFrame.from_dict(cat_quality_report)

,Feature,Count,% Miss.,Cardinality,Mode,Mode Freq.,Mode %,2nd Mode,2nd Mode Freq.,2nd Mode %
0,job,24318,0.6,12,JobCat3,5197,21.4,JobCat6,5053,20.8
1,marital,24318,0.0,3,married,14639,60.2,single,6885,28.3
2,education,24318,4.0,4,secondary,12516,51.5,tertiary,7065,29.1
3,default,24318,0.0,2,no,23871,98.2,yes,447,1.8
4,housing,24318,0.0,2,yes,13528,55.6,no,10790,44.4
5,loan,24318,0.0,2,no,20350,83.7,yes,3968,16.3
6,contact,24318,29.0,3,cellular,15691,64.5,NaN,7051,29.0
7,day,24318,0.0,31,20,1503,6.2,18,1261,5.2
8,month,24318,0.0,12,may,7448,30.6,jul,3751,15.4
9,poutcome,24318,81.3,4,NaN,19762,81.3,failure,2730,11.2


### Cleaning data
- Dropped `poutcome` and `contact` due to high missing values
- Filled in other missing values in features using the feature's mode

In [5]:
# dropped_headers = ["poutcome", "contact"]
dropped_headers = ["poutcome"]
df["job"] = df["job"].fillna(cat_quality_report["Mode"][0])
df["education"] = df["education"].fillna(cat_quality_report["Mode"][2])
df["contact"] = df["contact"].fillna(cat_quality_report["Mode"][6])
# df.drop(dropped_headers)
df = df.drop(dropped_headers, axis=1)


In [6]:
df

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,output
0,TR1,44,JobCat9,single,secondary,no,29,yes,no,cellular,5,may,0,1,-1,0,TypeA
1,TR2,31,JobCat4,married,secondary,no,2,yes,yes,cellular,5,may,0,1,-1,0,TypeA
2,TR3,42,JobCat4,divorced,tertiary,yes,2,yes,no,cellular,5,may,0,1,-1,0,TypeA
3,TR4,58,JobCat2,married,primary,no,121,yes,no,cellular,5,may,0,1,-1,0,TypeA
4,TR5,43,JobCat9,single,secondary,no,593,yes,no,cellular,5,may,0,1,-1,0,TypeA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24313,TR24314,36,JobCat9,married,secondary,no,557,yes,no,cellular,16,nov,0,4,-1,0,TypeB
24314,TR24315,53,JobCat6,married,tertiary,no,583,no,no,cellular,17,nov,0,1,184,4,TypeB
24315,TR24316,23,JobCat9,single,secondary,no,505,no,yes,cellular,17,nov,0,2,-1,0,TypeB
24316,TR24317,51,JobCat9,married,tertiary,no,825,no,no,cellular,17,nov,0,3,-1,0,TypeB


## Continuous Quality Report

## Creating categorical quality report

In [7]:
cont_df = df[cont_headers].copy()
cont_df

,age,balance,duration,campaign,pdays,previous
0,44,29,0,1,-1,0
1,31,2,0,1,-1,0
2,42,2,0,1,-1,0
3,58,121,0,1,-1,0
4,43,593,0,1,-1,0
...,...,...,...,...,...,...
24313,36,557,0,4,-1,0
24314,53,583,0,1,184,4
24315,23,505,0,2,-1,0
24316,51,825,0,3,-1,0


In [8]:
cont_df = df[cont_headers].copy()
cont_quality_report = create_cont_quality_report(cont_df, cont_headers)
cont_report_df = pd.DataFrame.from_dict(cont_quality_report)
cont_report_df

,Feature,Count,% Miss.,Card.,Min,1st Qrt.,Mean,Median,3rd Qrt.,Max,Std. Dev
0,age,24318,0.0,75,16,31.0,39.907723,37.0,48.00,95,11.438238
1,balance,24318,0.0,5635,-8019,75.0,1347.709968,451.0,1420.25,81204,2944.383929
2,duration,24318,0.0,1,0,0.0,0.000000,0.0,0.00,0,0.000000
3,campaign,24318,0.0,42,1,1.0,2.769060,2.0,3.00,63,3.068752
4,pdays,24318,0.0,478,-1,-1.0,41.085945,-1.0,-1.00,842,100.490570
5,previous,24318,0.0,37,0,0.0,0.591126,0.0,0.00,58,1.976166


## Cleaning
- Dropping duration as all values are 0

In [9]:
cont_dropped_headers = ["duration"]
df = df.drop(cont_dropped_headers, axis=1)
lower, upper = calc_clamps(df["campaign"])

temp = df["campaign"].copy()

for i in range(0, len(temp)):
    if temp.iloc[i] < lower:
        temp.iloc[i] = lower
    elif temp.iloc[i] > upper:
        temp.iloc[i] = upper
    # END IF
# END FOR

df["campaign"] = temp
# print(f"min: {temp.min()}, max: {temp.max()}")

In [10]:
lower, upper = calc_clamps(df["balance"])
print(lower, upper)

-601.5 2096.75


In [11]:
def create_binned_feature(feature):
    freqs = getEqualFreq(feature, 10)
    feature_binned = feature.copy()
    bins = {}
    
    for index, freq in enumerate(freqs):
        bins[freq] = f"bin{index}"
    
    for i in feature.index:
        for index, freq in enumerate(freqs):
            if freq >= feature.iloc[i]:
                feature_binned.iloc[i] = f"bin{index}"
                break
            # END IF
        # END FOR
    # END FOR
        
    return feature_binned, bins
# END create_bins_for_feature()

In [12]:
binned_features = {}
for cont_feature in cont_headers:
    if cont_feature in cont_dropped_headers:
        continue
    # END IF
    binned_series, bins = create_binned_feature(df[cont_feature])
    binned_features[cont_feature] = {"series": binned_series, "bins": bins}
# END FOR

for key, value in binned_features.items():
    df[key] = value["series"]
# END FOR

In [13]:
queries_df = pd.read_csv("queries.txt", names=headers, na_values="unknown")
queries_df.iloc[0]
# print(queries_df.iloc[0])

def run_naive_bayes_on_query(df_to_query, query, skip_processing=False):
    processed_query = query
    if skip_processing == False:
        series_dict = {}
        for index in query.index:
            if index == "id" or index == "output":
                continue
            # END IF

            if index in binned_features.keys():
                sorted_bins = sorted(binned_features[index]["bins"].items(), key=lambda x: x[0])
                for value, bin_str in sorted_bins:
                    if value > query[index]:
                        # print(f"index: {index}, bin: {bin_str}, bin value: {value}")
                        series_dict[index] = bin_str
                        break
                    # END IF
                # END FOR
            else:
                series_dict[index] = query[index]
            # END IF
        # END FOR
        processed_query = pd.Series(series_dict).drop(dropped_headers)
    # END IF
    
    df_a = df_to_query[df_to_query["output"] == "TypeA"]
    df_b = df_to_query[df_to_query["output"] == "TypeB"]
    a_prob = get_prob(df_to_query["output"], "TypeA")
    b_prob = get_prob(df_to_query["output"], "TypeB")
    # print(query.index)
    for index in query.index:
        a_prob = a_prob * get_prob(df_a[index], processed_query[index])
        b_prob = b_prob * get_prob(df_b[index], processed_query[index])
    # END FOR
    
    return "TypeA" if a_prob > b_prob else "TypeB"
# END run_naive_bayes_on_query

In [69]:
from multiprocessing import Process, Lock

df_copy = df.copy()

training_df = df_copy[:len(df_copy) - 5000]
testing_df = df_copy[len(df_copy) - 4999:]
print_mutex = Lock()

def run_naive_bayes_classifier(start, end, id):
    print(f"{id}: running from {start} to {end}")
    correct = 0
    
    for i in range(start, end):
        curr_query = testing_df.iloc[-i]
        results = run_naive_bayes_on_query(training_df, curr_query, skip_processing=True)
        
        if results == curr_query["output"]:
            correct = correct + 1
        # END IF
    # END FOR
    
    with print_mutex:
        print(f"{id} results")
        print(f"correct: {correct}, total: {end - start}")
        print(f"correct %: {(correct/(end - start)) * 100}")
    # END WITH
# END run_process

processes = []

id = 0
for i in range(0, 1750 + 1, 250):
    p = Process(target=run_naive_bayes_classifier, args=[1 if i == 0 else i, i + 250, id])
    processes.append(p)
    p.start()
    id = id + 1
# END FOR

for p in processes:
    p.join()
# END FOR

0: running from 1 to 250
1: running from 250 to 500
2: running from 500 to 750
3: running from 750 to 1000
4: running from 1000 to 1250
5: running from 1250 to 1500
6: running from 1500 to 1750
7: running from 1750 to 2000
6 results
correct: 250, total: 250
correct %: 100.0
2 results
correct: 250, total: 250
correct %: 100.0
5 results
correct: 250, total: 250
correct %: 100.0
0 results
correct: 249, total: 249
correct %: 100.0
4 results
correct: 250, total: 250
correct %: 100.0
1 results
correct: 250, total: 250
correct %: 100.0
7 results
correct: 250, total: 250
correct %: 100.0
3 results
correct: 250, total: 250
correct %: 100.0


In [66]:
def get_smooth_prob(feature, k=1):
    return (0 + k) / (len(feature) + (len(feature.value_counts()) * k))
# # END get_smooth_prob()

def get_prob(feature, value):
    feature_len = len(feature)
    counts = feature.value_counts(dropna=False)
    try:
        return counts[value] / feature_len
    except KeyError:
        return get_smooth_prob(feature)
    except IndexError:
        print(f"feature: {feature.name}, value: {value}")
    # END TRY
# END get_prob()